In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, median_absolute_error
from xgboost import XGBRegressor

/Users/machadolluis/miniforge3/envs/mlenv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [21]:
# Load data from Database into pandas DataFrame
df_X = pd.read_sql("select budget,popularity from clean_kaggle", con=engine)
df_y = pd.read_sql("select revenue from clean_kaggle", con=engine)

In [ ]:
# Split data features/targets into train and test
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, random_state=42)

In [22]:
# Features
df_X.shape

(500, 5)

In [23]:
# Predictor Variable
df_y.shape

(500,)

## Linear Regression Model

In [24]:
# Initiate the model
model = LinearRegression().fit(X_train, y_train)

# Start Prediction
y_pred = model.predict(X_test)

In [25]:
# Check model scores
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)
# median absolute error
medae = median_absolute_error(y_test, y_pred)
# r2 score
score = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Median Absolute Error: {medae}")
print(f"r-squared Score: {score}")

Mean Absolute Error: 2.7589452623437296
r-squared Score: 0.9992434463263903


## Ridge Regression Model

In [11]:
# Initiate the model
model = Ridge().fit(X_train, y_train)

# Start Prediction
y_pred = model.predict(X_test)

In [ ]:
# Check model scores
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)
# median absolute error
medae = median_absolute_error(y_test, y_pred)
# r2 score
score = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Median Absolute Error: {medae}")
print(f"r-squared Score: {score}")

Mean Absolute Error: 2.7589452623437296
r-squared Score: 0.9992434463263903


## Bayesian Ridge Regression Model

In [13]:
# Initiate the model
model = BayesianRidge().fit(X_train, y_train)

# Start Prediction
y_pred = model.predict(X_test)

In [ ]:
# Check model scores
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)
# median absolute error
medae = median_absolute_error(y_test, y_pred)
# r2 score
score = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Median Absolute Error: {medae}")
print(f"r-squared Score: {score}")

Mean Absolute Error: 2.7589452623437296
r-squared Score: 0.9992434463263903


## Random Forest Regression Model

In [15]:
# Initiate model 
model = RandomForestRegressor().fit(X_train,y_train)

# Start Prediction
y_pred = model.predict(X_test)

In [ ]:
# Check model scores
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)
# median absolute error
medae = median_absolute_error(y_test, y_pred)
# r2 score
score = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Median Absolute Error: {medae}")
print(f"r-squared Score: {score}")

Mean Absolute Error: 2.7589452623437296
r-squared Score: 0.9992434463263903


## XGBRegressor Model

In [17]:
# Initiate model
model = XGBRegressor().fit(X_train,y_train)

# Start Prediction
y_pred = model.predict(X_test)

In [ ]:
# Check model scores
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)
# median absolute error
medae = median_absolute_error(y_test, y_pred)
# r2 score
score = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Median Absolute Error: {medae}")
print(f"r-squared Score: {score}")

Mean Absolute Error: 2.7589452623437296
r-squared Score: 0.9992434463263903


## Final Model

After comparing each model's scores, we choose one for the final prediction.

In [ ]:
# final_model = model(random_state = 16).fit(X_train, y_train)

In [ ]:
# Predict values
y_pred_final = final_model.predict(X_test)